In [1]:
import jieba
from collections import Counter
import jieba.posseg as psg
jieba.load_userdict(r'stop_words/SWs.txt')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Kane\AppData\Local\Temp\jieba.cache
Loading model cost 0.654 seconds.
Prefix dict has been built successfully.


In [2]:
sequence = '新型冠状病毒'
# sequence = sequence.replace(' ','')

In [3]:
sequence.lower()

'新型冠状病毒'

In [4]:
nerRt = [(word, tag) for word, tag in psg.cut(sequence)]

In [5]:
nerRt

[('新型冠状病毒', 'x')]

In [6]:
spe = ['新冠','新型冠状','2019-nCoV','covid2019','新冠病毒']
if sequence in spe:
    print(sequence)

In [7]:
[(word,tag) for word, tag in psg.cut(sequence)]

[('新型冠状病毒', 'x')]

In [10]:
search_words_list = []
wordsList = sequence.replace("+"," ") #查询的+会被urlEncode成%2B,所以不用特殊处理+字符
wordsList = [x for x in jieba.cut_for_search(''.join(wordsList)) if len(x)>1] #长度小于2的自动过滤掉，暂时认为是无意义的
# wordsList = Counter(wordsList)
# wordsList[0][0]
# for word in wordsList:
#             search_words_list.append(word[0])
# print(search_words_list)
wordsList


# 高亮
# for info in result["hits"]["hits"]:
#             for word in search_words_list:
#                 info["title"].replace(str(word),'<span class="highlighted">'+word+'</span>')
#                 info["body"].replace(str(word), '<span class="highlighted">' + word + '</span>')

['新型', '冠状', '病毒', '新型冠状病毒']

In [13]:
from py2neo import Graph, Node, Relationship, NodeMatcher, RelationshipMatcher
from py2neo.data import Node, Relationship

url = 'localhost:7474'
username = 'neo4j'
# password = 'neo4j'
password = 'kaneyd..369'

graph = Graph(url, username=username, password=password)

def searchAll(label, value):
    rt = graph.run("match (s:`{}`)-[p]->(o) where s.value=~'.*{}.*' return type(p), o.value, labels(o)".format(label, value)).data()
    if len(rt) < 6:
        return -1
    else:
        property_dict = {}
        for po in rt:
            if po['type(p)'] not in property_dict:
                property_dict[po['type(p)']] = [(po['labels(o)'][0],po['o.value']),]
            else:
                property_dict[po['type(p)']].append((po['labels(o)'][0],po['o.value']))
        return property_dict

In [22]:
# graph
from py2neo import Graph

url = 'localhost:7474'
username = 'neo4j'
# password = 'neo4j'
password = 'kaneyd..369'
graph = Graph(url, username=username, password=password)
value = '钟南山'
pkLabel = graph.run("match (s) where s.value= '{}' return labels(s)".format(value)).data()
rtTo = graph.run("match (s)-[p]->(o) where s.value= '{}' return type(p), o.value, labels(o)".format(value)).data()
rtFrom = graph.run("match (s)<-[p]-(o) where s.value= '{}' return type(p), o.value, labels(o)".format(value)).data()
property_dictTo = {}
for po in rtTo:
    if po['type(p)'] not in property_dictTo:
        property_dictTo[po['type(p)']] = [(po['labels(o)'][0], po['o.value']), ]
    else:
        property_dictTo[po['type(p)']].append((po['labels(o)'][0], po['o.value']))
        
property_dictFrom = {}
for po in rtFrom:
    if po['type(p)'] not in property_dictFrom:
        property_dictFrom[po['type(p)']] = [(po['labels(o)'][0], po['o.value']), ]
    else:
        property_dictFrom[po['type(p)']].append((po['labels(o)'][0], po['o.value']))
# pkLabel[0]['labels(s)']
pkLabel

[{'labels(s)': ['人物']}]

In [85]:
property_dictTo = property_dictFrom
property_dictTo

{'相关疾病': [('战役', '新型冠状病毒战役')], '导致': [('病毒', '新型冠状病毒 2019-nCoV')]}

In [88]:
property_dictFrom

{'相关疾病': [('战役', '新型冠状病毒战役')], '导致': [('病毒', '新型冠状病毒 2019-nCoV')]}

In [90]:

rtTo1 = graph.run("match (s)-[p]->(o) where s.value= '{}' return type(p), o.value, labels(o)".format('新型冠状病毒战役')).data()
rtTo2 = graph.run("match (s)-[p]->(o) where s.value= '{}' return type(p), o.value, labels(o)".format('新型肺炎')).data()
# rtFrom = graph.run("match (s)<-[p]-(o) where s.value= '{}' return type(p), o.value, labels(o)".format(value)).data()
rt = rtTo1 + rtTo2
print(rtTo1)
print(rtTo2)
rt

[{'type(p)': '爆发地点', 'o.value': '武汉', 'labels(o)': ['城市']}, {'type(p)': '相关疾病', 'o.value': '新型肺炎', 'labels(o)': ['疾病']}]
[]


[{'type(p)': '爆发地点', 'o.value': '武汉', 'labels(o)': ['城市']},
 {'type(p)': '相关疾病', 'o.value': '新型肺炎', 'labels(o)': ['疾病']}]

In [92]:
value = ['新型肺炎','新型冠状病毒战役']
rtTo1 = graph.run("match (s)<-[p]-(o) where s.value= '{}' return type(p), o.value, labels(o)".format('新型冠状病毒战役')).data()
rtTo2 = graph.run("match (s)<-[p]-(o) where s.value= '{}' return type(p), o.value, labels(o)".format(v)).data() for in value
rt = rtTo1 + rtTo2
print(rtTo1)
print(rtTo2)
rt

[{'type(p)': '参与战役', 'o.value': '张继先', 'labels(o)': ['人物']}, {'type(p)': '参与战役', 'o.value': '钟南山', 'labels(o)': ['人物']}, {'type(p)': '参与战役', 'o.value': '张定宇', 'labels(o)': ['人物']}, {'type(p)': '参与战役', 'o.value': '李兰娟', 'labels(o)': ['人物']}, {'type(p)': '参与战役', 'o.value': '张文宏', 'labels(o)': ['人物']}]
[{'type(p)': '相关疾病', 'o.value': '新型冠状病毒战役', 'labels(o)': ['战役']}, {'type(p)': '导致', 'o.value': '新型冠状病毒 2019-nCoV', 'labels(o)': ['病毒']}]


[{'type(p)': '参与战役', 'o.value': '张继先', 'labels(o)': ['人物']},
 {'type(p)': '参与战役', 'o.value': '钟南山', 'labels(o)': ['人物']},
 {'type(p)': '参与战役', 'o.value': '张定宇', 'labels(o)': ['人物']},
 {'type(p)': '参与战役', 'o.value': '李兰娟', 'labels(o)': ['人物']},
 {'type(p)': '参与战役', 'o.value': '张文宏', 'labels(o)': ['人物']},
 {'type(p)': '相关疾病', 'o.value': '新型冠状病毒战役', 'labels(o)': ['战役']},
 {'type(p)': '导致', 'o.value': '新型冠状病毒 2019-nCoV', 'labels(o)': ['病毒']}]

In [103]:
values = ['新型肺炎','新型冠状病毒战役']
rtTo = []
for v in values: 
    rtTo += graph.run("match (s)<-[p]-(o) where s.value= '{}' return type(p), o.value, labels(o)".format(v)).data()
property_dictTo = {}
for po in rtTo:
    if po['type(p)'] not in property_dictTo:
        property_dictTo[po['type(p)']] = [(po['labels(o)'][0], po['o.value']), ]
    else:
        property_dictTo[po['type(p)']].append((po['labels(o)'][0], po['o.value']))

In [104]:
property_dictTo['别名'] = [('Concept String', n) for n in ['新冠', '新型冠状', '2019-nCoV', 'covid2019', '新冠病毒']]
property_dictTo

{'相关疾病': [('战役', '新型冠状病毒战役')],
 '导致': [('病毒', '新型冠状病毒 2019-nCoV')],
 '参与战役': [('人物', '张继先'),
  ('人物', '钟南山'),
  ('人物', '张定宇'),
  ('人物', '李兰娟'),
  ('人物', '张文宏')],
 '别名': [('Concept String', '新冠'),
  ('Concept String', '新型冠状'),
  ('Concept String', '2019-nCoV'),
  ('Concept String', 'covid2019'),
  ('Concept String', '新冠病毒')]}

In [102]:
[('Concept String', n) for n in ['新冠', '新型冠状', '2019-nCoV', 'covid2019', '新冠病毒']]

[('Concept String', '新冠'),
 ('Concept String', '新型冠状'),
 ('Concept String', '2019-nCoV'),
 ('Concept String', 'covid2019'),
 ('Concept String', '新冠病毒')]